In [1]:
import numpy as np
import pandas as pd

train_path = "X_train.csv"
label_path = "y_train.csv"
test_path = "X_test.csv"
samples_path = "samples.csv"

df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
labeldf = pd.read_csv(label_path)


labels = labeldf.values
data = df.values
test_data = test_df.values

data = data[:,1:]
test_data = test_data[:,1:]
labels = labels[:,1]


print(data.shape)
print(test_data.shape)
print(labels.shape)

(5117, 17813)
(3411, 17813)
(5117,)


In [2]:
def count_instances(labels):
    unique, counts = np.unique(labels,return_counts=True)
    return dict(zip(unique,counts))

class_counts = count_instances(labels)

print("Class 0 Counts: {}".format(class_counts[0]))
print("Class 1 Counts: {}".format(class_counts[1]))
print("Class 2 Counts: {}".format(class_counts[2]))
print("Class 3 Counts: {}".format(class_counts[3]))


(num_train, max_timesteps) = data.shape
(num_test, _) = test_data.shape

Class 0 Counts: 3030
Class 1 Counts: 443
Class 2 Counts: 1474
Class 3 Counts: 170


In [3]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(1)

Using TensorFlow backend.
/home/wen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wen/.local/lib/python3.6/site-packages/tensorflow/python/framework/d

In [4]:
# # truncate and pad input sequences
max_review_length = data.shape[1]
# X_train = sequence.pad_sequences(data, maxlen=max_review_length, padding='post')
# X_test = sequence.pad_sequences(test_data, maxlen=max_review_length, padding='post')
X_train = np.nan_to_num(data, nan = 0)
X_test = np.nan_to_num(test_data,nan=0)

In [5]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [6]:
X_train.shape

(5117, 1, 17813)

In [7]:
# create and fit the LSTM network
import keras

labels = keras.utils.to_categorical(labels)

number_of_classes = 4
batch_size = 64
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(1, 17813)))
# model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.25))
model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.25))
model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.25))
model.add(LSTM(32))
model.add(Dense(number_of_classes, activation='softmax'))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=50, verbose=1, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, labels, epochs=250, batch_size=batch_size, validation_split=0.2, verbose=2, shuffle=False, callbacks=[early_stopping])
# model.save('Keras_models/my_model_' + str(i) + '_' + str(j) + '_' + str() + '.h5')
# predictions = model.predict(X_val)
# score = accuracy_score(change(Y_val), change(predictions))
# print(score)


Train on 4093 samples, validate on 1024 samples
Epoch 1/250


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(64, 512), b.shape=(512, 2048), m=64, n=2048, k=512
	 [[{{node lstm_1/while/MatMul_1}}]]
	 [[Mean/_203]]
  (1) Internal: Blas GEMM launch failed : a.shape=(64, 512), b.shape=(512, 2048), m=64, n=2048, k=512
	 [[{{node lstm_1/while/MatMul_1}}]]
0 successful operations.
0 derived errors ignored.